In [3]:
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,238 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [47.7 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,698 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvi

In [ ]:
filepath = r'/content/template_1.csv'

In [ ]:
import time
import csv
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Словарь для отображения кодов регионов в читаемые названия
REGION_NAMES = {
    "100000000": "область Абай",
    "110000000": "Акмолинская область",
    "150000000": "Актюбинская область",
    "190000000": "Алматинская область",
    "230000000": "Атырауская область",
    "270000000": "Западно-Казахстанская область",
    "310000000": "Жамбылская область",
    "330000000": "область Жетісу",
    "350000000": "Карагандинская область",
    "390000000": "Костанайская область",
    "430000000": "Кызылординская область",
    "470000000": "Мангистауская область",
    "550000000": "Павлодарская область",
    "590000000": "Северо-Казахстанская область",
    "610000000": "Туркестанская область",
    "620000000": "область Ұлытау",
    "630000000": "Восточно-Казахстанская область",
    "710000000": "г. Астана",
    "750000000": "г. Алматы",
    "790000000": "г. Шымкент"
}

def parse_codes_from_raw_csv(filepath):
    """
    Считывает CSV (template_1.csv) и возвращает список кортежей (код, название).
    Формат строки: 'A02.055.000;Консультация: Стоматолог-терапевт'
    """
    parsed = []
    with open(filepath, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            # Пропускаем заголовок
            if i == 0:
                continue
            parts = line.strip().replace('"', '').split(";")
            if len(parts) == 2:
                code = parts[0].strip().lstrip("\t")
                name = parts[1].strip()
                parsed.append((code, name))
    return parsed

def get_region_values():
    """
    Открывает сайт и извлекает из выпадающего списка все value (коды регионов),
    чтобы в дальнейшем их перебрать.
    """
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=options)
    driver.get("https://fms.ecc.kz/ru/fsms/services_registry")
    wait = WebDriverWait(driver, 10)

    region_select = Select(wait.until(
        EC.presence_of_element_located((By.NAME, "region"))
    ))
    region_codes = [
        opt.get_attribute("value")
        for opt in region_select.options
        if opt.get_attribute("value")
    ]
    driver.quit()
    return region_codes

def save_result(filepath, row):
    """
    Сохраняет строку в CSV (append-режим).
    """
    with open(filepath, mode="a", newline='', encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(row)

def load_checked_pairs(filepath):
    """
    Считывает уже сохранённые пары (Регион, Код)
    чтобы не парсить повторно, если мы перезапустим парсер.
    """
    checked = set()
    if not os.path.exists(filepath):
        return checked

    with open(filepath, "r", encoding="utf-8") as file:
        reader = csv.reader(file)
        next(reader, None)  # Пропустить заголовок
        for row in reader:
            if len(row) >= 2:
                region_col = row[0]       # Название региона
                code_col = row[1].split(" - ")[0]  # Из 'A02.055.000 - Консультация' берём 'A02.055.000'
                checked.add((region_col, code_col))
    return checked

def process_one(region_code, code, name):
    """
    Обрабатывает 1 комбинацию (регион, услуга).
    Возвращает [<регион>, <код - название>, <таблица>] или None.
    """
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    driver = webdriver.Chrome(options=options)
    driver.get("https://fms.ecc.kz/ru/fsms/services_registry")
    wait = WebDriverWait(driver, 15)

    region_name = REGION_NAMES.get(region_code, region_code)

    try:
        # Выбор региона
        region_select = Select(wait.until(
            EC.presence_of_element_located((By.NAME, "region"))
        ))
        region_select.select_by_value(region_code)

        # Ввод кода услуги
        q_input = driver.find_element(By.ID, "q")
        q_input.clear()
        q_input.send_keys(code)

        # Нажимаем 'Найти'
        search_btn = driver.find_element(By.XPATH, "//button[contains(text(), 'Найти')]")
        search_btn.click()

        # Ждём появления блока content-block
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "content-block")))
        time.sleep(1.0)  # небольшая пауза, чтобы страница точно обновилась

        # Смотрим, есть ли таблица с поставщиками
        page_html = driver.page_source
        soup = BeautifulSoup(page_html, "html.parser")
        table = soup.find("table")

        if not table:
            # Значит, услуга не найдена
            print(f"Пропущено (нет таблицы): {region_name} | {code}")
            return None

        # Если таблица есть, превращаем её в текст
        table_text = table.get_text(separator=" ", strip=True)
        print(f"✅ Найдено: {region_name} | {code}")
        return [region_name, f"{code} - {name}", table_text]

    except Exception as e:
        print(f"❌ Ошибка: {region_name} | {code} — {e}")
        return None
    finally:
        driver.quit()

def main():
    # Шаг 1: Загружаем услуги
    codes_list = parse_codes_from_raw_csv("template_1.csv")

    # Шаг 2: Получаем все коды регионов
    all_region_codes = get_region_values()

    # Шаг 3: Готовим CSV-файл
    output_file = "результаты_поиска.csv"
    if not os.path.exists(output_file):
        with open(output_file, mode="w", newline='', encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(["Регион", "Код или название услуги", "Результат (текст)"])

    # Шаг 4: Загружаем уже обработанные пары, чтобы не парсить повторно
    checked_pairs = load_checked_pairs(output_file)

    # Шаг 5: Многопоточность
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []

        for region_code in all_region_codes:
            for code, name in codes_list:
                region_name = REGION_NAMES.get(region_code, region_code)
                # Если уже есть, пропускаем
                if (region_name, code) in checked_pairs:
                    continue
                # Создаём задачу
                futures.append(
                    executor.submit(process_one, region_code, code, name)
                )

        # Шаг 6: Обработка результатов
        for fut in as_completed(futures):
            result = fut.result()
            if result:  # если парсинг дал результат
                save_result(output_file, result)

if __name__ == "__main__":
    main()

📄 Собираем данные со страницы: 1
📄 Собираем данные со страницы: 2
📄 Собираем данные со страницы: 3
📄 Собираем данные со страницы: 4
📄 Собираем данные со страницы: 5
📄 Собираем данные со страницы: 6
📄 Собираем данные со страницы: 7
📄 Собираем данные со страницы: 8
📄 Собираем данные со страницы: 9
📄 Собираем данные со страницы: 10
📄 Собираем данные со страницы: 11
📄 Собираем данные со страницы: 12
📄 Собираем данные со страницы: 13
📄 Собираем данные со страницы: 14
📄 Собираем данные со страницы: 15
📄 Собираем данные со страницы: 16
📄 Собираем данные со страницы: 17
📄 Собираем данные со страницы: 18
📄 Собираем данные со страницы: 19
📄 Собираем данные со страницы: 20
📄 Собираем данные со страницы: 21
📄 Собираем данные со страницы: 22
📄 Собираем данные со страницы: 23
📄 Собираем данные со страницы: 24
📄 Собираем данные со страницы: 25
📄 Собираем данные со страницы: 26
📄 Собираем данные со страницы: 27
📄 Собираем данные со страницы: 28
📄 Собираем данные со страницы: 29
📄 Собираем данные со ст

,Страница,№ лота,Заказчик,Наименование объявления,Наименование ЛС и ИМН,Характеристика,Способ закупки,Планируемый срок закупки (месяц),Кол-во единиц измерения,Цена за единицу (тенге),Сумма,Статус,Кол-во заявок
0,1,4662097-Зму1,"НАО ""Фонд социального медицинского страхования""",[2025 год – ГОБМП и ОСМС] Объявление о проведе...,Медицинская помощь онкогематологическим больным,Медицинская помощь онкогематологическим больным,Закуп медицинских услуг,,3 173.00,1 178 037.82,986 404.35,Опубликован (прием заявок),0
1,1,4661904-Зму1,"НАО ""Фонд социального медицинского страхования""",[2025 год – ГОБМП и ОСМС] Объявление о проведе...,Специализированная медицинская помощь в стацио...,Специализированная медицинская помощь в стацио...,Закуп медицинских услуг,,291 269.00,150 395.12,272 230 296.47,Опубликован (прием заявок),0
2,1,4661824-Зму2,"НАО ""Фонд социального медицинского страхования""",[2025 год – ГОБМП и ОСМС] Объявление о проведе...,Оказание медицинской помощи на уровне первично...,Оказание медицинской помощи на уровне первично...,Закуп медицинских услуг,,1 206 115.00,23 499.20,44 274 991.19,Опубликован (прием заявок),0
3,1,4661961-Зму1,"НАО ""Фонд социального медицинского страхования""",[2025 год – ГОБМП и ОСМС] Объявление о проведе...,Патологоанатомическая диагностика,Патологоанатомическая диагностика,Закуп медицинских услуг,,23 566.00,11 828.91,22 564 060.54,Опубликован (прием заявок),0
4,1,4661941-Зму1,"НАО ""Фонд социального медицинского страхования""",[2025 год – ГОБМП и ОСМС] Объявление о проведе...,Паллиативная медицинская помощь,Паллиативная медицинская помощь,Закуп медицинских услуг,,45 080.00,13 067.19,200 634 000.00,Опубликован (прием заявок),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43644,4365,4440851-ДТ1,"ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",Закуп лекарственных средств и изделий медицинс...,Амоксициллин****,таблетка 250мг,Двухэтапный тендер,Август,27 144.00,9.92,269 268.48,Закупка не состоялась,0
43645,4365,4440850-ДТ1,"ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",Закуп лекарственных средств и изделий медицинс...,Лекарственное средство факторов свертывания кр...,"Лиофилизат для приготовления раствора, 500 МЕ",Двухэтапный тендер,Август,6 978 432.00,46.35,323 450 323.20,Закупка не состоялась,0
43646,4365,4440849-ДТ1,"ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",Закуп лекарственных средств и изделий медицинс...,Амитриптилина гидрохлорид,раствор для инъекций 20мг/2мл,Двухэтапный тендер,Август,1 080.00,25.40,27 432.00,Закупка не состоялась,0
43647,4365,4440848-ДТ1,"ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",Закуп лекарственных средств и изделий медицинс...,Трифлуоперазин****,"раствор для инъекций 0,2%, 1 мл",Двухэтапный тендер,Август,76 150.00,33.55,2 554 832.50,Закупка не состоялась,0
